In [1]:
import os
import time
import shutil
import pickle

import torch
import torch.nn.functional as F

from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
# from tensorboard_logger import configure, log_value

from model import RecurrentAttention
from utils import AverageMeter

from torchvision import transforms, utils, models
from PIL import Image

import torch

import utils
import data_loader

from trainer import Trainer
from config import get_config

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import pandas as pd

import json

In [2]:
def load_inputs(impath):
    to_tens = transforms.ToTensor()
    return to_tens(Image.open(impath).convert('RGB')).unsqueeze(0)

In [3]:
m = open("../../pooling/data/migration_data.json",)
mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
q = 2
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data

,muni_id,num_migrants,class
0,484001001,42055.0,1
1,484001002,4017.0,1
2,484001003,11992.0,1
3,484001004,762.0,1
4,484001005,7551.0,1
...,...,...,...
2326,484032049,2487.0,1
2327,484032050,2024.0,1
2328,484032051,3084.0,1
2329,484032052,2919.0,1


In [4]:
mig_data['class'].value_counts()

0    1166
1    1165
Name: class, dtype: int64

In [5]:
def get_png_names(directory):
    images = []
    for i in os.listdir(directory):
        try:
            if os.path.isdir(os.path.join(directory, i)):
                new_path = os.path.join(directory, i, "pngs")
                image = os.listdir(new_path)[0]
                images.append(os.path.join(directory, i, "pngs", image))
        except:
            pass
    return images


            

image_names = get_png_names("../../pooling/data/MEX/")

y_class, y_mig = [], []

for i in image_names:
        dta = mig_data[mig_data["muni_id"] == i.split("/")[5]]
        if len(dta) != 0:
            y_class.append(dta['class'].values[0])
            y_mig.append(dta['num_migrants'].values[0])

In [6]:
import matplotlib.pyplot as plt
import torchvision

In [7]:
import random

train_num = int(25 * .70)

train_indices = random.sample(range(0, 25), train_num)
val_indices = [i for i in range(0, 25) if i not in train_indices]

In [8]:
import torchvision

batch_size = 1

# train = [(load_inputs(image_paths[i]).squeeze()[:, 0:28, 0:28], ys[i]) for i in range(0, 93)]
# val = [(load_inputs(image_paths[i]).squeeze()[:, 0:28, 0:28], ys[i]) for i in range(93, 133)]

# brighten = torchvision.transforms.functional.adjust_brightness(brightness_factor = 2)


train = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in train_indices]
val = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in val_indices]


train_dl = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = True)

In [9]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  17
Num validation:  8


In [10]:
import numpy as np
from utils import plot_images

import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [11]:
config, unparsed = get_config()

In [12]:
trainer = Trainer(config, (train_dl, val_dl))

In [13]:
trainer.train()

  0%|          | 0/17 [00:00<?, ?it/s]/rapids/notebooks/sciclone/geograd/Heather/RAM/continuous_v2/modules.py:375: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  a_t_class = F.log_softmax(self.fc_class1(h_t)).flatten(start_dim = 0) # shape goes from [2,2] -> [1,4]
/rapids/notebooks/sciclone/geograd/Heather/RAM/continuous_v2/modules.py:379: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  a_t_class = F.log_softmax(self.fc_class2(a_t_class)).unsqueeze(0)
0.1s - loss: 0.943 - acc: 0.000 - mae: 13.716:   0%|          | 0/17 [00:00<?, ?it/s]


[*] Train on 17 samples, validate on 8 samples

Epoch: 1/200 - LR: 0.000300


1.4s - loss: 124.582 - acc: 100.000 - mae: 11991.670: 100%|██████████| 17/17 [00:01<00:00, 12.14it/s]


train loss: 19.430 - train acc: 58.824 - val loss: 80.410 - val acc: 62.500 - val err: 37.500- train mae: 1616.324 - val mae: 7738.806 [*]


0.2s - loss: 12.966 - acc: 100.000 - mae: 954.667:  12%|█▏        | 2/17 [00:00<00:01, 11.10it/s]

...

train loss: 17.169 - train acc: 58.824 - val loss: 78.399 - val acc: 62.500 - val err: 37.500- train mae: 1615.805 - val mae: 7738.288

Epoch: 27/200 - LR: 0.000300


0.9s - loss: 0.092 - acc: 0.000 - mae: 46.142: 100%|██████████| 17/17 [00:00<00:00, 19.90it/s]      
0.0s - loss: 12.415 - acc: 100.000 - mae: 987.176:   0%|          | 0/17 [00:00<?, ?it/s]

train loss: 17.270 - train acc: 58.824 - val loss: 78.963 - val acc: 62.500 - val err: 37.500- train mae: 1615.758 - val mae: 7738.298

Epoch: 28/200 - LR: 0.000300


0.5s - loss: 42.357 - acc: 100.000 - mae: 4016.198:  41%|████      | 7/17 [00:00<00:00, 12.28it/s]


KeyboardInterrupt: 

In [23]:
test = torch.rand(2, 256)
patch_dict = {}

for patch in range(0, test.shape[0]):
    cur_vals = list(test[patch].numpy())
    patch_dict[patch] = [str(i) for i in cur_vals]
    
with open("sample.json", "w") as outfile: 
    json.dump(patch_dict, outfile)